## 1. Fixing Forecasted Values for Independent Features with Poor Performance:
This is done to get the validation metrics for each forecasted independent feature. Those features with poor MAPE (>0.4) will be modelled again using Poisson Distribution and Exponential Smoothing - Holt-Winters Method. We will then take the forecasted values of the model with minimum MAPE value.

In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from statsmodels.discrete.count_model import Poisson
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt

In [2]:
forecasts_data_dict = {}
val_data_dict = {}

csv_names = ['engine_1',
 'engine_2',
 'engine_3',
 'engine_5',
 'engine_7',
 'engine_8',
 'engine_9',
 'engine_10_truck_2',
 'engine_12',
 'engine_13_truck_10',
 'engine_16_truck_6',
 'engine_17_rescue_11',
 'engine_19',
 'truck_3',
 'truck_4',
 'truck_5']

for i in csv_names:
    globals()[i+'_fc'] = pd.read_csv(f'/Users/medhinisridharr/Documents/University of Rochester/Courses/Fall 2024/Capstone Project/monthly_incident_counts/Forecasted Selected Features/Past_Future/{i}_past_future.csv')
    globals()[i+'_val'] = pd.read_csv(f'/Users/medhinisridharr/Documents/University of Rochester/Courses/Fall 2024/Capstone Project/monthly_incident_counts/Forecasted Selected Features/validation_metrics/{i}_fc_val.csv')
    
    if 'time_to_reach' in globals()[i+'_fc']:
        globals()[i+'_fc'].drop('time_to_reach',axis=1,inplace=True)
        
    if 'time_to_reach' in globals()[i+'_val']:
        globals()[i+'_val'].drop('time_to_reach',axis=1,inplace=True)
        
    forecasts_data_dict[i+'_fc'] = globals()[i+'_fc']
    
    if 'Unnamed: 0' in globals()[i+'_val'].columns:
        globals()[i+'_val'].rename(columns={'Unnamed: 0':'feature'},inplace=True)
        
    val_data_dict[i+'_val'] = globals()[i+'_val']

In [3]:
for val_df in list(val_data_dict.keys())[:8]:
    cols = list(globals()[val_df].columns)
    globals()[val_df]['feature'] = ['RMSE','MAE','MAPE','AIC','BIC']
    globals()[val_df] = globals()[val_df][['feature']+cols]
    val_data_dict[val_df] = globals()[val_df]

In [4]:
val_data_transpose_dict = {}
for k in val_data_dict.keys():
    globals()[k+'_t'] = val_data_dict[k].T.reset_index()
    globals()[k+'_t'].columns = globals()[k+'_t'].loc[0]
    assert list(globals()[k+'_t'].columns) == ['feature', 'RMSE', 'MAE', 'MAPE', 'AIC', 'BIC']
    globals()[k+'_t'] = globals()[k+'_t'].loc[1:].reset_index(drop=True)
    val_data_transpose_dict[k] = globals()[k+'_t']

- Created the improve_forecast() to look at poorly forecasted features and run Poisson Distribution and ETS models to get alternate forecasts. The one with least MAPE is chosen and the forecasted values are replaced in the data before predicting monthly counts based on this.

In [5]:
def improve_forecast(df, df_val, df_val_t, poor_forecast):
    print(poor_forecast)
    
    df.rename(columns={'datetimealarm_month_year':'ds'},inplace=True)
    sarimax_forecasts = df.copy()
    
    final_forecasts = sarimax_forecasts[['ds']].copy()  # Include the datetime column
    
    for feature in poor_forecast:
        print(f"Processing feature: {feature}")

        # Extract the data for the feature
        feature_data = sarimax_forecasts[['ds', feature]].copy()
        feature_data[feature] = feature_data[feature].apply(lambda x: max(0, x))  # Ensure no negative values

        # Split data into train (historical) and test (validation) for MAPE calculation
        train_data = feature_data[feature_data['ds']<"2023-01"][feature]  # Historical data
        test_data = feature_data[(feature_data['ds'] >= "2023-01") & (feature_data['ds'] <= "2024-09")][feature]  # Jan 2023 to Sep 2024

        # SARIMAX MAPE (validation on test_data)
        sarimax_mape = df_val.loc[df_val['feature']=='MAPE',feature].tolist()[0]
        print(f"SARIMAX MAPE for {feature}: {sarimax_mape}")

        # Initialize alternative model forecast
        alternative_forecast_poisson = None
        alternative_mape_poisson = None

        alternative_forecast_ets = None
        alternative_mape_ets = None

        # Prepare training data for Poisson
        y_train = train_data.values
        X_train = np.arange(len(y_train)).reshape(-1, 1)  # Example: Simple index as regressor
        
        X_test = np.arange(len(y_train), len(y_train) + len(test_data)).reshape(-1, 1)

        # Fit Poisson model
        poisson_model = Poisson(y_train, X_train).fit(disp=False)
        alternative_forecast_poisson = poisson_model.predict(X_test)
        alternative_mape_poisson = mean_absolute_percentage_error(test_data, alternative_forecast_poisson)
        print(f"Alternative Model MAPE for {feature}: {alternative_mape_poisson}")

        # Fit ETS model
        ets_model = ExponentialSmoothing(
            train_data,
            seasonal="add",  # Example: Additive seasonality
            seasonal_periods=12  # Example: Monthly seasonality
        ).fit()

        # Calculate MAPE for the alternative model
        alternative_forecast_ets = ets_model.forecast(len(test_data))
        alternative_mape_ets = mean_absolute_percentage_error(test_data, alternative_forecast_ets)
        print(f"Alternative Model MAPE for {feature}: {alternative_mape_ets}")

        best_mape = min([alternative_mape_poisson,alternative_mape_ets,sarimax_mape])
        
        y_full_train = feature_data[feature_data['ds'] < "2024-10"][feature].values
        X_full_train = np.arange(len(y_full_train)).reshape(-1, 1)
        X_future = np.arange(len(y_full_train), len(y_full_train) + 120).reshape(-1, 1)

        if best_mape == alternative_mape_poisson:
            print(f"Using Poisson model forecast for {feature}")
            poisson_model = Poisson(y_full_train, X_full_train).fit(disp=False)
            alternative_forecast_poisson = poisson_model.predict(X_future)
            final_forecasts[feature] = df.loc[df['ds'] <= "2024-09",feature].tolist() + list(np.round(alternative_forecast_poisson).astype(int))
            df_val_t.loc[df_val_t['feature']==feature,'MAPE'] = best_mape
            
        elif best_mape == alternative_mape_ets:
            print(f"Using ETS model forecast for {feature}")
            alternative_forecast_ets = ets_model.forecast(120)
            final_forecasts[feature] = df.loc[df['ds'] <= "2024-09",feature].tolist() + list(np.round(alternative_forecast_ets).astype(int))
            df_val_t.loc[df_val_t['feature']==feature,'MAPE'] = best_mape
        
        else:
            print(f"Using SARIMAX forecast for {feature}")
            final_forecasts[feature] = sarimax_forecasts[feature]

    
    good_features = [i for i in df.columns if i not in poor_forecast + ['ds']]
    final_forecasts = pd.merge(final_forecasts, sarimax_forecasts[['ds'] + good_features], on='ds', how='left')
    assert len(final_forecasts) == 333, len(final_forecasts)
    return final_forecasts, df_val_t

In [6]:
final_forecasts_data_dict = {}
for (f,v) in zip(forecasts_data_dict.keys(),val_data_dict.keys()):
    print(f+': ')
    
    test_val_t = val_data_transpose_dict[v]
    p=[i for i in test_val_t.loc[test_val_t['MAPE']>40,'feature'].tolist() if i!='is_covid']
    
    globals()['final_'+f], globals()[v+'_t']  = improve_forecast(forecasts_data_dict[f], val_data_dict[v], val_data_transpose_dict[v],p)
    final_forecasts_data_dict[f] = globals()['final_'+f]
    val_data_transpose_dict[v] = globals()[v+'_t']
    print('-'*80)
    print('')

engine_1_fc: 
['hour_of_day_6', 'hour_of_day_3']
Processing feature: hour_of_day_6
SARIMAX MAPE for hour_of_day_6: inf
Alternative Model MAPE for hour_of_day_6: 3192142741470494.5
Alternative Model MAPE for hour_of_day_6: 2051010677818874.8
Using ETS model forecast for hour_of_day_6
Processing feature: hour_of_day_3
SARIMAX MAPE for hour_of_day_3: 48.3153175013337
Alternative Model MAPE for hour_of_day_3: 0.8741685546592988
Alternative Model MAPE for hour_of_day_3: 0.4250239441217267
Using ETS model forecast for hour_of_day_3
--------------------------------------------------------------------------------

engine_2_fc: 
['hour_of_day_8']
Processing feature: hour_of_day_8
SARIMAX MAPE for hour_of_day_8: 44.7383389694951
Alternative Model MAPE for hour_of_day_8: 0.9629157617923325
Alternative Model MAPE for hour_of_day_8: 0.23949193324497203
Using ETS model forecast for hour_of_day_8
--------------------------------------------------------------------------------

engine_3_fc: 
['civilia

/Users/medhinisridharr/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/medhinisridharr/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/medhinisridharr/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Alternative Model MAPE for hour_of_day_4: 0.45994458442734626
Using ETS model forecast for hour_of_day_4
--------------------------------------------------------------------------------

engine_17_rescue_11_fc: 
['hour_of_day_6', 'hour_of_day_10', 'hour_of_day_12', 'hour_of_day_2']
Processing feature: hour_of_day_6
SARIMAX MAPE for hour_of_day_6: 53.21344465603146
Alternative Model MAPE for hour_of_day_6: 1.827757029581272
Alternative Model MAPE for hour_of_day_6: 0.31846446072251167
Using ETS model forecast for hour_of_day_6
Processing feature: hour_of_day_10
SARIMAX MAPE for hour_of_day_10: 78.50984050186239
Alternative Model MAPE for hour_of_day_10: 3.2476422056178054
Alternative Model MAPE for hour_of_day_10: 0.2646301318728474
Using ETS model forecast for hour_of_day_10
Processing feature: hour_of_day_12
SARIMAX MAPE for hour_of_day_12: 48.55163566674763
Alternative Model MAPE for hour_of_day_12: 3.129892893974284
Alternative Model MAPE for hour_of_day_12: 0.2653315195521573
Using

In [24]:
for k in val_data_dict.keys():
    print(k)

engine_1_val
engine_2_val
engine_3_val
engine_5_val
engine_7_val
engine_8_val
engine_9_val
engine_10_truck_2_val
engine_12_val
engine_13_truck_10_val
engine_16_truck_6_val
engine_17_rescue_11_val
engine_19_val
truck_3_val
truck_4_val
truck_5_val


In [25]:
for (k,v) in val_data_transpose_dict.items():
    v.to_csv(f'/Users/medhinisridharr/Documents/University of Rochester/Courses/Fall 2024/Capstone Project/monthly_incident_counts/Forecasted Selected Features/corrected_validation_metrics/{k}_cor_val.csv',index=False)

## Predicting Monthly Incident Counts using Prophet

In [8]:
#Create eval metrics table
prophet_metrics_df = pd.DataFrame(columns=['station_name','MAE','RMSE','MAPE'])
prophet_metrics_df['station_name'] = csv_names
prophet_metrics_df

,station_name,MAE,RMSE,MAPE
0,engine_1,NaN,NaN,NaN
1,engine_2,NaN,NaN,NaN
2,engine_3,NaN,NaN,NaN
3,engine_5,NaN,NaN,NaN
4,engine_7,NaN,NaN,NaN
5,engine_8,NaN,NaN,NaN
6,engine_9,NaN,NaN,NaN
7,engine_10_truck_2,NaN,NaN,NaN
8,engine_12,NaN,NaN,NaN
9,engine_13_truck_10,NaN,NaN,NaN


### Prophet Model:
- Training: Train on data till year = 2023 (inclusive)
    - Add regressors and then train
- Test: Test on January 2024 - September 2024
- Calculate evaluation metrics
- Train on entire data
    - Add regressors and then train
- Predict for future periods: October 2024 - September 2034
- Using final forecasted values for monthly incident counts, plot data from 2007 to 2034.

In [9]:
for (n,v) in zip(csv_names,final_forecasts_data_dict.values()):
    v['ds'] = pd.to_datetime(v['ds'])
    train = v[v['ds'].dt.year <= 2023].reset_index(drop=True)
    test = v[(v['ds'].dt.year > 2023) & (v['ds']<= "2024-09")].reset_index(drop=True)

    # Prepare data for Prophet (rename columns as Prophet expects specific column names)
    train_data = train.rename(columns={'monthly_incidents_per_nearest_station_count': 'y'})
    test_data = test.rename(columns={'monthly_incidents_per_nearest_station_count': 'y'})

    #Add indepenent features (historical + forecasted values) as regressors in the Prophet model
    regressors_list = [i for i in train_data.columns if i not in ['ds','y']]

    missing_columns = [col for col in regressors_list if col not in test_data.columns]
    assert len(missing_columns)==0, f"Missing columns are {missing_columns}"

    # Initialize and train the Prophet model
    model = Prophet()

    for regressor in regressors_list:
        model.add_regressor(regressor)

    model.fit(train_data)

    # Make future dataframe for test period
    future = test_data[['ds']].copy()
    for regressor in regressors_list:
        future[regressor] = test_data[regressor]

    # Predict using the trained model
    forecast = model.predict(future)

    # Evaluate the model
    actuals = test_data['y'].values
    predictions = forecast['yhat'].values

    # Calculate evaluation metrics
    mae = mean_absolute_error(actuals, predictions)
    rmse = mean_squared_error(actuals, predictions, squared=False)
    mape = (np.abs(actuals - predictions) / actuals).mean() * 100

    #Add metrics in the table
    prophet_metrics_df.loc[prophet_metrics_df['station_name']==n,'MAE'] = mae
    prophet_metrics_df.loc[prophet_metrics_df['station_name']==n,'RMSE'] = rmse
    prophet_metrics_df.loc[prophet_metrics_df['station_name']==n,'MAPE'] = mape

    plt.figure(figsize=(10, 6))
    plt.plot(test_data['ds'], actuals, label='Actuals', marker='o')
    plt.plot(test_data['ds'], predictions, label='Forecast', marker='x')
    plt.title(f"Prophet Model for {n}: Forecast vs Actuals")
    plt.xlabel("Date")
    plt.ylabel("Monthly Incident Counts")
    plt.legend()
    plt.savefig(f'/Users/medhinisridharr/Documents/University of Rochester/Courses/Fall 2024/Capstone Project/monthly_incident_counts/Predicted Monthly Incident Counts/Plots - Forecast vs Actuals/{n}_forecast_vs_actuals.png', format='png', dpi=300)  # Save the plot as a PNG file with high resolution
    plt.close()

    # Combine train and test data into a single dataset
    full_data = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)
    
    forecasted_values = v[v['ds']>= "2024-10"].reset_index(drop=True)

    for feature in forecasted_values.columns:
        if feature not in ['ds','monthly_incidents_per_nearest_station_count']:
            #Preprocessing - if SARIMAX has predicted negative counts and this is more than 50% - drop, otherwise, force to 0
            if len(forecasted_values[forecasted_values[feature] < 0]) / len(forecasted_values) > 0.5:
                forecasted_values = forecasted_values.drop(feature, axis=1)
                regressors_list.remove(feature)
                full_data = full_data.drop(feature, axis=1)
            else:
                forecasted_values[feature] = np.where(forecasted_values[feature] < 0, 0, forecasted_values[feature])
            
                # Round forecasted values to the nearest integer since they are counts
                forecasted_values[feature] = forecasted_values[feature].round().astype(int)

    # Initialize the Prophet model
    final_model = Prophet(changepoint_prior_scale=0.5, uncertainty_samples=3000)

    # Add regressors to the model
    for regressor in regressors_list:
        final_model.add_regressor(regressor)

    # Fit the model on the full historical data
    final_model.fit(full_data)

    # Create a DataFrame for future predictions (next 10 years)
    # Ensure the future DataFrame has columns for all regressors
    future_periods = pd.date_range(start="2024-10-01", end="2034-09-01", freq='MS')  # Monthly start
    future = pd.DataFrame({'ds': future_periods})

    # Add forecasted regressor values for the future period
    for regressor in regressors_list:
        # Replace with your forecasted regressor data
        future[regressor] = forecasted_values[regressor]  # Ensure `forecasted_values` is defined

    # Predict using the trained model
    final_forecast = final_model.predict(future)

    # Amplify uncertainty intervals for better visualization
#     amplification_factor = 2  # Adjust this factor as needed
#     final_forecast['yhat_lower_vis'] = final_forecast['yhat'] - amplification_factor * (final_forecast['yhat'] - final_forecast['yhat_lower'])
#     final_forecast['yhat_upper_vis'] = final_forecast['yhat'] + amplification_factor * (final_forecast['yhat_upper'] - final_forecast['yhat'])

    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(full_data['ds'], full_data['y'], label='Historical Data', marker='o', linestyle='-')
    plt.plot(final_forecast['ds'], final_forecast['yhat'], label='Forecast (2024-2034)', marker='x', linestyle='--')

    # Plot the uncertainty interval
    plt.fill_between(final_forecast['ds'], 
                     final_forecast['yhat_lower'], 
                     final_forecast['yhat_upper'], 
                     color='blue', 
                     alpha=0.2, 
                     label='Uncertainty Interval')
    plt.title(f"{n}: Monthly Incident Counts Forecast (2024-2034)")
    plt.xlabel("Date")
    plt.ylabel("Monthly Incident Counts")
    plt.legend()
    plt.savefig(f'/Users/medhinisridharr/Documents/University of Rochester/Courses/Fall 2024/Capstone Project/monthly_incident_counts/Predicted Monthly Incident Counts/Plots - Future Counts/{n}_monthly_counts_forecast.png', format='png', dpi=300)  # Save the plot as a PNG file with high resolution
    plt.close()
    
    # Combine historical and forecasted data
    historical_data = full_data.copy()  # Historical data (2007-2024)
    historical_data = historical_data[['ds', 'y'] + regressors_list]  # Include 'ds', 'y', and regressors
    historical_data = historical_data.rename(columns={'y': 'monthly_incidents_per_nearest_station_count'})

    forecast_data = final_forecast[['ds', 'yhat'] + [f'{regressor}' for regressor in regressors_list]].copy()
    forecast_data = forecast_data.rename(columns={'yhat': 'monthly_incidents_per_nearest_station_count'})

    # Combine historical and forecasted data
    combined_data = pd.concat([historical_data, forecast_data], axis=0).reset_index(drop=True)

    # Ensure the date range is from Jan 2007 to Sep 2034
    date_range = pd.date_range(start="2007-01-01", end="2034-09-01", freq='MS')
    final_combined_data = pd.DataFrame({'ds': date_range}).merge(combined_data, on='ds', how='left')

    # Save to CSV
    final_combined_data.to_csv(f'/Users/medhinisridharr/Documents/University of Rochester/Courses/Fall 2024/Capstone Project/monthly_incident_counts/Predicted Monthly Incident Counts/Forecasted Data/{n}_historical_forecasted_values.csv', index=False)


20:10:45 - cmdstanpy - INFO - Chain [1] start processing
20:10:45 - cmdstanpy - INFO - Chain [1] done processing
20:10:46 - cmdstanpy - INFO - Chain [1] start processing
20:10:46 - cmdstanpy - INFO - Chain [1] done processing
20:10:46 - cmdstanpy - INFO - Chain [1] start processing
20:10:46 - cmdstanpy - INFO - Chain [1] done processing
20:10:46 - cmdstanpy - INFO - Chain [1] start processing
20:10:47 - cmdstanpy - INFO - Chain [1] done processing
20:10:47 - cmdstanpy - INFO - Chain [1] start processing
20:10:47 - cmdstanpy - INFO - Chain [1] done processing
20:10:47 - cmdstanpy - INFO - Chain [1] start processing
20:10:48 - cmdstanpy - INFO - Chain [1] done processing
20:10:48 - cmdstanpy - INFO - Chain [1] start processing
20:10:48 - cmdstanpy - INFO - Chain [1] done processing
20:10:48 - cmdstanpy - INFO - Chain [1] start processing
20:10:48 - cmdstanpy - INFO - Chain [1] done processing
20:10:49 - cmdstanpy - INFO - Chain [1] start processing
20:10:49 - cmdstanpy - INFO - Chain [1]

In [10]:
prophet_metrics_df

,station_name,MAE,RMSE,MAPE
0,engine_1,0.291661,0.460196,0.142126
1,engine_2,0.781801,0.820729,0.20228
2,engine_3,0.140351,0.190015,0.077823
3,engine_5,0.474015,0.489339,0.260761
4,engine_7,0.184456,0.209461,0.091862
5,engine_8,0.014136,0.01694,0.036455
6,engine_9,0.295105,0.402123,0.108305
7,engine_10_truck_2,0.642628,0.670564,0.222747
8,engine_12,0.34545,0.7522,0.534572
9,engine_13_truck_10,0.391101,0.471319,0.159016


In [12]:
print('Avg. MAE across stations: '+str(round(prophet_metrics_df['MAE'].mean(),3)))
print('Avg. RMSE across stations: '+str(round(prophet_metrics_df['RMSE'].mean(),3)))
print('Avg. MAPE across stations: '+str(round(prophet_metrics_df['MAPE'].mean(),3)))

Avg. MAE across stations: 0.34
Avg. RMSE across stations: 0.43
Avg. MAPE across stations: 0.192


In [13]:
print('Median MAE across stations: '+str(round(prophet_metrics_df['MAE'].median(),3)))
print('Median RMSE across stations: '+str(round(prophet_metrics_df['RMSE'].median(),3)))
print('Median MAPE across stations: '+str(round(prophet_metrics_df['MAPE'].median(),3)))

Median MAE across stations: 0.359
Median RMSE across stations: 0.471
Median MAPE across stations: 0.162


In [26]:
prophet_metrics_df.to_csv('prophet_metrics.csv',index=False)

In [31]:
prophet_metrics_df[['MAE', 'RMSE', 'MAPE']] = prophet_metrics_df[['MAE', 'RMSE', 'MAPE']].astype(float).round(3)
prophet_metrics_df

,station_name,MAE,RMSE,MAPE
0,engine_1,0.292,0.460,0.142
1,engine_2,0.782,0.821,0.202
2,engine_3,0.140,0.190,0.078
3,engine_5,0.474,0.489,0.261
4,engine_7,0.184,0.209,0.092
5,engine_8,0.014,0.017,0.036
6,engine_9,0.295,0.402,0.108
7,engine_10_truck_2,0.643,0.671,0.223
8,engine_12,0.345,0.752,0.535
9,engine_13_truck_10,0.391,0.471,0.159
